In [1]:
#Joseph K Blankenship, Pramiti Barua, Jaswitha Ravala
#CS 5050 Project: Sentiment Classification of Amazon Reviews with Hybrid Naive Bayes Approach

In [2]:

import pandas as pd
import re
import nltk
import numpy as np
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.corpus import sentiwordnet as swn
from sklearn.model_selection import train_test_split
#from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score,classification_report
from sklearn.feature_extraction.text import CountVectorizer


In [3]:
nltk.download('wordnet')
ar = pd.read_csv("Amazon_Consumer_Reviews_of_Amazon_Products.csv")

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\JoeKa\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:
#nltk.download('sentiwordnet')

In [5]:
#nltk.download('averaged_perceptron_tagger')

In [6]:
#nltk.download('punkt')

In [7]:
stop = stopwords.words('english')

In [8]:
new_stop = ["i","you","he","she","it","we","they","them","us","him", "me", "her"]
stop_no_pro = []
for word in stop:
    if word in new_stop:
        continue 
    else:
        stop_no_pro.append(word)

In [9]:
#stop_no_pro = nltk.tokenize(stop_no_pro)

In [10]:
review_list_reg = []
for review in ar['reviews.text']:
     review_list_reg.append(review)

In [11]:
#This is function to generate the normalized senti-score for our reviews.

def senti_calc(word, tag):
    pos_score = 0
    neg_score = 0
    
    if 'NN' in tag and len(list(swn.senti_synsets(word, 'n')))>0:
        syn_set = list(swn.senti_synsets(word, 'n'))
    elif 'VB' in tag and len(list(swn.senti_synsets(word, 'v')))>0:
        syn_set = list(swn.senti_synsets(word, 'v'))
    elif 'JJ' in tag and len(list(swn.senti_synsets(word, 'a')))>0:
        syn_set = list(swn.senti_synsets(word, 'a'))
    elif 'RB' in tag and len(list(swn.senti_synsets(word, 'r')))>0:
        syn_set = list(swn.senti_synsets(word, 'r'))
    elif 'PRP' in tag:
        #neg_score += .5
        pos_score += .5
    else:
        return (0,0)
    
    if ('PRP' in tag):
        return (pos_score, neg_score)
    else:
        for syn in syn_set:
            pos_score += syn.pos_score()
            neg_score += syn.neg_score()
        return (pos_score/len(syn_set), neg_score/len(syn_set))



In [12]:
pos_senti_score = []
neg_senti_score = []
for review in review_list_reg:
    #no_punct = tokenizer.tokenize(review)
    tokens = nltk.word_tokenize(review)
    tags = nltk.pos_tag(tokens)
    length = len(tags)
    pos_score = 0
    neg_score = 0
    for word,tag in tags:
        pos_score += senti_calc(word,tag)[0]
        neg_score += senti_calc(word,tag)[1]
    pos_senti_score.append(pos_score/length)
    neg_senti_score.append(neg_score/length)


In [13]:
review_rating = []
for rating in ar['reviews.rating']:
     review_rating.append(rating)

In [14]:
classifier = MultinomialNB()

In [15]:
vectorizer = CountVectorizer(stop_words = stop_no_pro)
X = vectorizer.fit_transform(review_list_reg)

C:\Users\JoeKa\anaconda3\envs\cs5050\lib\site-packages\sklearn\feature_extraction\text.py:386: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['it', 'she', 'you'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [16]:
matrix = X.toarray()

In [17]:
length = len(pos_senti_score)
for x in range(length):
    pos_senti_score[x] = int(pos_senti_score[x] * 100)

In [18]:
length = len(neg_senti_score)
for x in range(length):
    neg_senti_score[x] = int(neg_senti_score[x] * 100)

In [19]:
matrix2 = np.c_[matrix,neg_senti_score]

In [20]:
matrix_f = np.c_[matrix2,pos_senti_score]

In [21]:
X_train, X_test, y_train, y_test = train_test_split(matrix_f, review_rating)

In [22]:
classifier.fit(X_train, y_train)

MultinomialNB()

In [23]:
y_pred = classifier.predict(X_test)

In [24]:
accuracy = accuracy_score(y_test,y_pred)

In [25]:
accuracy

0.6832

In [26]:
report = classification_report(y_test,y_pred)

In [27]:
print(report)

              precision    recall  f1-score   support

           1       0.50      0.09      0.15        11
           2       1.00      0.08      0.15        12
           3       0.50      0.03      0.05        70
           4       0.40      0.28      0.33       318
           5       0.75      0.91      0.82       839

    accuracy                           0.68      1250
   macro avg       0.63      0.28      0.30      1250
weighted avg       0.64      0.68      0.64      1250

